In [ ]:
using JuMP
using CPLEX

# Ejemplo ULS

Recordemos que el modelo que tenemos es:

\begin{align}
\text{minimize} \qquad & \sum_{t\in T} c_s s_t+ \sum_{t\in T} c_l y_t \\
 \text{subject to} \quad \quad & s_{t-1} + x_t = s_t + d_t \\
 \qquad \qquad & x_t \leq M y_t \\
 \qquad \qquad & s_0 = 0 \\
 \qquad \qquad & x_t, s_t \in \mathbb{R}^+ \forall t\in T\\
 \qquad \qquad & y_t \in \{0,1\}
\end{align}


In [ ]:
T=10
d=[5; 10; 7; 11; 13; 14; 2; 8; 17; 21]
cl=50
cs=2
M=sum(d);

In [ ]:
modelo=Model(solver = CplexSolver())
@variable(modelo, x[1:T]>= 0)
@variable(modelo, y[1:T],Bin)
@variable(modelo, s[0:T]>=0)

#función objetivo a minimizar
@objective(modelo, Min, sum(cs*s[t] for t in 1:T)+sum(cl*y[t] for t in 1:T))

for t in 1:T
    @constraint(modelo,s[t-1]+x[t]==d[t]+s[t])
end
for t in 1:T
    @constraint(modelo,x[t]<=M*y[t])
end
@constraint(modelo,s[0]==0)

#escribimos el modelo
print(modelo)

#resolvemos
status = solve(modelo)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(modelo))
println("**** x = ", getvalue(x))
println("**** y = ", getvalue(y))
println("**** s = ", getvalue(s))

## Alternativa leer datos de Excel

Aquí también es posible leer datos de Excel. Hay que utilizar un paquete llamado ExcelReaders

In [ ]:
#Pkg.add("ExcelReaders") 
#C:\Users\Jordi\AppData\Local\JuliaPro-0.6.1.1\Python>python -m pip install xlrd

using ExcelReaders

In [ ]:
valor=readxl("uls_data.xlsx","Sheet1!B3")
T=convert(Int8,valor)
cl=readxl("uls_data.xlsx","Sheet1!B1")
cs=readxl("uls_data.xlsx","Sheet1!B2")
ultimaFila=5+T
rango="Sheet1!B6:B$ultimaFila"
d=readxl("uls_data.xlsx",rango);
M=sum(d);

In [ ]:
# Modificar el comportamiento del solver (p.ej. eliminar información por pantalla)

#creamos un modelo
m = Model(solver = CplexSolver(CPX_PARAM_SCRIND=0))
@variable(m, x[1:T]>= 0)
@variable(m, y[1:T],Bin)
@variable(m, s[0:T]>=0)

#función objetivo a minimizar
@objective(m, Min, sum(cs*s[t] for t in 1:T)+sum(cl*y[t] for t in 1:T))

for t in 1:T
    @constraint(m,s[t-1]+x[t]==d[t]+s[t])
end
for t in 1:T
    @constraint(m,x[t]<=M*y[t])
end
@constraint(m,s[0]==0)

#escribimos el modelo
#print(m)

#resolvemos
status = solve(m)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(modelo))
println("**** x = ", getvalue(x))
println("**** y = ", getvalue(y))
println("**** s = ", getvalue(s))

# También podemos añadir cortes. 

Por ejemplo, en este problema sabemos que la solución óptima cumple con este conjunto de desigualdades:

\begin{equation}
\sum_{j\in L\setminus S}x_j + \sum_{j\in S}d_{jl} y_{j} \geq d_{1l},\qquad \forall L\in \{1,...,l\}, S\subseteq L
\end{equation}

donde:

\begin{equation}
d_{jl}=\sum_{t=j}^{l} d_t
\end{equation}

que indica que la demanda total entre los periodos $j$ y $l$.

Esta restricción es relativamente fácil de separar dada una solución fraccionaria

In [ ]:
function separate(T::Int64, sumd::Array{Float64, 2}, y_val, x_val, y, x)
    TOL=1E-6
    S = zeros(Bool, T)
    for l in 1:T
        fill!(S, false)
        lhsvalue = 0.  #x(L\S) + sum{d[j:l]*y[j] for j in S}
        empty = true
        for j in 1:l
            if x_val[j] > sumd[j,l]*y_val[j] + TOL
                S[j] = true
                empty = false
                lhsvalue += sumd[j,l]*y_val[j]
            else
                lhsvalue += x_val[j]
            end
        end
        if empty #fuerza que haya algo en S, si no la igualdad es trivial
            continue
        end
        if lhsvalue < sumd[1,l] - TOL
            lhs = sum(x[1:l])
            for j = (1:T)[S]
                lhs += sumd[j,l]*y[j] - x[j]
            end
            return lhs - sumd[1,l] #para retornar mayor que 0
        end
    end
    return
end

function readULSdesdeExcel(fichero::String)
    valor=readxl(fichero,"Sheet1!B3")
    T=convert(Int64,valor)
    cl=readxl(fichero,"Sheet1!B1")
    cs=readxl(fichero,"Sheet1!B2")
    ultimaFila=5+T
    rango="Sheet1!B6:B$ultimaFila"
    d=readxl(fichero,rango);
    return T,cs,cs,d
end

function solveULS(fichero::String, solver=CplexSolver(), valid::Bool = true)

    T, cs, cl, d = readULSdesdeExcel(fichero)
    m = Model(solver = solver)
    @variable(m, y[1:T], Bin)
    @variable(m, x[i = 1:T] >= 0)
    @variable(m, s[1:T] >= 0)

    @objective(m, Min, sum(cl*y[t] + cs*s[t] for t in 1:T))
    @constraint(m, activation[t = 1:T], x[t] <= sum(d[t:T])*y[t])
    @constraint(m, balance[t = 1:T], (t>1?s[t-1]:0) + x[t] == s[t] + d[t])

    #recomputo sum(d[j:l])
    sumd = zeros(Float64, T, T)
    for l = 1:T, j = 1:l
        sumd[j,l] = sum(d[j:l])
    end
    
    separationtime = 0.
    separated = 0
    called = 0
    function lSgenerator(cb)
        tt = time()
        called += 1
        y_val = getvalue(y)
        x_val = getvalue(x)
     
        expr = separate(T, sumd, y_val, x_val, y, x)
        if expr != nothing
            @usercut(cb, expr >= 0)
            separated += 1
        end
        separationtime += time()-tt
    end
    if valid
        addcutcallback(m, lSgenerator)
    end
    status = solve(m)
    println("Objective value: ", getobjectivevalue(m))
    println("Separation time: $separationtime seconds")
    println("Separated: $separated")
    status
end

In [ ]:
#solveULS("uls_data.xlsx", solver=CplexSolver(CPX_PARAM_CUTSFACTOR=1))
solveULS("uls_data.xlsx") 

# Integración con Excel

La clave para integrar con Excel es llamar a julia desde Excel, guardar los resultados en un archivo y leerlo y escribirlo. No es la forma más elegante pero funciona.

Iremos por pasos:

* Escribir la rutina en un archivo .jl
* Crear la macro que llama el archivo .jl
* Leer el archivo generado por julia y guardarlo en la hoja de Excel